In [ ]:
!pip install thop

In [1]:
import time
import json
import copy
import torch
import numpy as np
import torch.nn as nn
from tqdm import tqdm
from PIL import Image
import seaborn as sns
from thop import profile
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score

from src.model import MiniCNN, ConvBlock, FCBlock
from src.train import trainModel
from src.data import CustomDataset, load_cifar_10_data, check_data_loading, Loader, class_to_idx
from src.visualizations import plotFmaps_and_activationHist, plotCurves
from src.utils import EarlyStopping, unpickle, loadWeights, readJson, genError, saveHistory, evalModel

In [ ]:
"""
from google.colab import drive
drive.mount('/content/drive')
"""

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
#base = "/content/drive/MyDrive/project1"
base = "/home/fw7th/regularization-ml"

In [ ]:
"""
# Copy once from Drive
!cp /content/drive/MyDrive/project1/cifar-10-python.tar.gz /content/

# Extract locally
!mkdir /content/dataset/
!tar -xvzf /content/cifar-10-python.tar.gz -C /content/dataset/
"""

In [ ]:
# for the second model, we'll experiment on the effects of cutout at a value of 0.3 in the fc layer
base_conv_layers = [
    ConvBlock(3, 64, pool=False),
    ConvBlock(64, 64),
    ConvBlock(64, 128, pool=False),
    ConvBlock(128, 128)
]

base_fc_layers = [
    FCBlock(128, 64, True, 0.3),
    nn.Linear(64, 10)
]

drop_3_model = MiniCNN(base_conv_layers, base_fc_layers)

In [ ]:
# we're using the same dataset, and model architecture so let's train
history_drop_3 = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": [], "last_epoch": 0, "LR": []}
dropoutfc0 = "dropoutfc0.3"
drop_03_trained = trainModel(drop_3_model, history_drop_3, train_loader, val_loader, dropoutfc0)
saveHistory(history_drop_3, dropoutfc0)